### Libraries and Data

In [11]:
%cd C:\Users\leoma\Desktop\projects\Modelos-de-Pronosticos-Series-Temporales-Python\Facebook Prophet

c:\Users\leoma\Desktop\projects\Modelos-de-Pronosticos-Series-Temporales-Python\Facebook Prophet


In [12]:
# import libraries
import itertools
from prophet import Prophet
import pandas as pd
import numpy as np

In [13]:
#load the data
data = pd.read_csv('DHS_Daily_Report_2020.csv')
data.head()

,Date,Total Adults in Shelter,Total Children in Shelter,Total Individuals in Shelter,Single Adult Men in Shelter,Single Adult Women in Shelter,Total Single Adults in Shelter,Families with Children in Shelter,Adults in Families with Children in Shelter,Children in Families with Children in Shelter,Total Individuals in Families with Children in Shelter,Adult Families in Shelter,Easter,Thanksgiving,Christmas
0,1/1/2014,28770,21919,50689,7269,2650,9919,10469,14897,21919,1866,3954,0,0,0
1,1/2/2014,28843,21954,50797,7289,2661,9950,10484,14930,21954,1871,3963,0,0,0
2,1/3/2014,28947,22007,50954,7323,2678,10001,10521,14978,22007,1872,3968,0,0,0
3,1/4/2014,28954,22003,50957,7320,2684,10004,10520,14971,22003,1877,3979,0,0,0
4,1/5/2014,28991,21966,50957,7386,2692,10078,10502,14941,21966,1874,3972,0,0,0


* we see first, the date is in the wrong format
* we have a lot of variables here, the one that I´ll use as depende variable will be as "Total I ndiviuals in Shelter" 
* "Easter", "Thanksgiving" and Christmas which I´ll have as regressors

### Data Preparation

In [14]:
# transform the date vatiable
data.Date = pd.to_datetime(data.Date, format = "%m/%d/%Y")
data.Date

0      2014-01-01
1      2014-01-02
2      2014-01-03
3      2014-01-04
4      2014-01-05
          ...    
2552   2020-12-27
2553   2020-12-28
2554   2020-12-29
2555   2020-12-30
2556   2020-12-31
Name: Date, Length: 2557, dtype: datetime64[ns]

In [15]:
# final dataset -- quiero dejar mis ultimos 31 dias
df = data.loc[data.Date <= "2020-12-01",
            ["Date", "Total Individuals in Shelter",
            "Easter", "Thanksgiving", "Christmas"]]
df.head()


,Date,Total Individuals in Shelter,Easter,Thanksgiving,Christmas
0,2014-01-01,50689,0,0,0
1,2014-01-02,50797,0,0,0
2,2014-01-03,50954,0,0,0
3,2014-01-04,50957,0,0,0
4,2014-01-05,50957,0,0,0


1. Rename Dependent and time Variable to "y" and "ds"

In [21]:
#RENOMBRAMOS LA VARIABLE QUE QUEREMOS PRONOSTICAR
df = df.rename(columns = {'Total Individuals in Shelter' : 'y', 
                                'Date' : 'ds'})
df.head(1)

,ds,y,Easter,Thanksgiving,Christmas
0,2014-01-01,50689,0,0,0


2. Declare Easter and Thanksgiving as holidays. Combine them. Use pd.concat

In [22]:
#easter
easter_dates = dataset[dataset.Easter == 1].ds
easter = pd.DataFrame({'holiday': 'easter',
                               'ds': pd.to_datetime(easter_dates),
                               'lower_windows': -5,
                               'upper_windows': 2})
easter.head(2)

,holiday,ds,lower_windows,upper_windows
109,easter,2014-04-20,-5,2
459,easter,2015-04-05,-5,2


In [23]:
#Thanksgiving
thanksgiving_dates = dataset[dataset.Thanksgiving == 1].ds
Thanksgiving = pd.DataFrame({'holiday': 'thanksgiving',
                               'ds': pd.to_datetime(thanksgiving_dates),
                               'lower_windows': -5,
                               'upper_windows': 2})
Thanksgiving.head(2)

,holiday,ds,lower_windows,upper_windows
330,thanksgiving,2014-11-27,-5,2
694,thanksgiving,2015-11-26,-5,2


In [24]:
#combine holidays
holidays = pd.concat([easter, Thanksgiving])
holidays

,holiday,ds,lower_windows,upper_windows
109,easter,2014-04-20,-5,2
459,easter,2015-04-05,-5,2
816,easter,2016-03-27,-5,2
1201,easter,2017-04-16,-5,2
1551,easter,2018-04-01,-5,2
1936,easter,2019-04-21,-5,2
2293,easter,2020-04-12,-5,2
330,thanksgiving,2014-11-27,-5,2
694,thanksgiving,2015-11-26,-5,2
1058,thanksgiving,2016-11-24,-5,2


3. Create Prophet model. Christmas is a regressor

In [25]:
m = Prophet(holidays = holidays,
            seasonality_mode="multiplicative", 
            seasonality_prior_scale=10, 
            holidays_prior_scale=10, 
            changepoint_prior_scale=0.05)
m.add_regressor('Christmas')
m.fit(df)

19:08:31 - cmdstanpy - INFO - Chain [1] start processing
19:08:32 - cmdstanpy - INFO - Chain [1] done processing


4. Cross-Validation. Horizon = 31, initial = 2400. 
Assess via MAE

In [26]:
#CV
from prophet.diagnostics import cross_validation
df_cv = cross_validation(m,
                         horizon='31 days', # cuantos días en el futuro pronosticamos?
                         initial= '2400 days', # a que dist. comenzamos la valid.cruzada? tiene sentido que siempre al menos dejemos un período completo
                         parallel= "processes")
df_cv.head()

,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,2020-07-31,56064.409895,55780.761155,56333.856099,54930,2020-07-30
1,2020-08-01,56048.226539,55793.589192,56300.499939,54810,2020-07-30
2,2020-08-02,56170.966714,55905.134326,56427.115355,54956,2020-07-30
3,2020-08-03,56226.185716,55966.346755,56496.605885,54978,2020-07-30
4,2020-08-04,56217.954028,55965.062102,56483.333360,54943,2020-07-30


In [28]:
# Assess model
from sklearn.metrics import mean_absolute_error
mean_absolute_error(df_cv.y, df_cv.yhat)

1996.86796116217

Si vemos algun ejemplo, 2k es un buen promedio respecto a los 50689 del 2014-01-01. De todas maneras es posible que podamos mejorarlo

5. Create Parameter Grid for Tuning

In [29]:
#Parameters
from sklearn.model_selection import ParameterGrid
param_grid = {'seasonality_prior_scale': [10, 20],
              'changepoint_prior_scale': [0.05, 0.1],
              'holidays_prior_scale': [10, 20],}
grid = ParameterGrid(param_grid)

6. Perform Hyperparameter Tuning. Use MAE as the KPI to Optimize.
Gather results

In [30]:
# Hyper parameter tuning
mae = []
for params in grid:
    #Build the model
    m = Prophet(holidays= holidays, 
            seasonality_mode="multiplicative", 
            seasonality_prior_scale=params['seasonality_prior_scale'],
            holidays_prior_scale=params['holidays_prior_scale'], 
            changepoint_prior_scale=params['changepoint_prior_scale'])
    m.add_regressor('Christmas')
    m.fit(df)

    #Cross-Validation (siempre para que no sea casualidad de que obtengamos un bueno o malo resultado)
        
    df_cv = cross_validation(m,
                         horizon='31 days', # cuantos días en el futuro pronosticamos?
                         initial= '2400 days', # a que dist. comenzamos la valid.cruzada? tiene sentido que siempre al menos dejemos un período completo
                         parallel= "processes")
    
    #Gather the results
    error = mean_absolute_error(df_cv.y, df_cv.yhat)
    mae.append(error)


19:24:39 - cmdstanpy - INFO - Chain [1] start processing
19:24:40 - cmdstanpy - INFO - Chain [1] done processing
19:24:49 - cmdstanpy - INFO - Chain [1] start processing
19:24:50 - cmdstanpy - INFO - Chain [1] done processing
19:24:59 - cmdstanpy - INFO - Chain [1] start processing
19:25:00 - cmdstanpy - INFO - Chain [1] done processing
19:25:10 - cmdstanpy - INFO - Chain [1] start processing
19:25:11 - cmdstanpy - INFO - Chain [1] done processing
19:25:20 - cmdstanpy - INFO - Chain [1] start processing
19:25:22 - cmdstanpy - INFO - Chain [1] done processing
19:25:30 - cmdstanpy - INFO - Chain [1] start processing
19:25:32 - cmdstanpy - INFO - Chain [1] done processing
19:25:41 - cmdstanpy - INFO - Chain [1] start processing
19:25:42 - cmdstanpy - INFO - Chain [1] done processing
19:25:51 - cmdstanpy - INFO - Chain [1] start processing
19:25:53 - cmdstanpy - INFO - Chain [1] done processing


In [31]:
# best parameters
best_params = grid[np.argmin(mae)]
print(best_params)

{'seasonality_prior_scale': 20, 'holidays_prior_scale': 20, 'changepoint_prior_scale': 0.1}
